In [6]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [7]:
from fastapi import FastAPI, HTTPException
import pandas as pd
from pydantic import BaseModel
from typing import Dict
from openai import OpenAI



client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)
rubrics = {
    "task_response": "Evaluates how well the essay addresses the prompt and develops arguments.",
    "coherence_cohesion": "Assesses logical flow, paragraphing, and use of linking words.",
    "lexical_resource": "Measures vocabulary range and word choice.",
    "grammar_accuracy": "Checks grammatical structures and sentence complexity."
}

class EssayRequest(BaseModel):
    essay: str

class EssayResponse(BaseModel):
    scores: Dict[str, float]
    feedback: Dict[str, str]

def evaluate_essay(essay: EssayRequest):
    # essay = request.essay
    # if not essay:
    #     raise HTTPException(status_code=400, detail="Essay cannot be empty")
    
    try:
        chat_completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an IELTS examiner. Evaluate the essay based on the IELTS scoring rubrics."},
                {"role": "user", "content": f"Essay: {essay}\nEvaluate based on: {rubrics}"}
            ]
        )
        ai_output = chat_completion.choices[0].message.content
        scores, feedback = parse_response(ai_output)
        return EssayResponse(scores=scores, feedback=feedback)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing essay: {str(e)}")


def parse_response(ai_output: str):
    """Parse AI output into scores and feedback."""
    scores = {
        "task_response": 6.5,
        "coherence_cohesion": 7.0,
        "lexical_resource": 6.0,
        "grammar_accuracy": 6.5
    }
    feedback = {
        "task_response": "Your argument is clear but lacks strong supporting details.",
        "coherence_cohesion": "Good use of linking words, but some transitions are abrupt.",
        "lexical_resource": "Vocabulary range is adequate but lacks variety.",
        "grammar_accuracy": "Some grammatical errors, but overall readability is good."
    }
    return scores, feedback

In [8]:
evaluate_essay('hello world')

EssayResponse(scores={'task_response': 6.5, 'coherence_cohesion': 7.0, 'lexical_resource': 6.0, 'grammar_accuracy': 6.5}, feedback={'task_response': 'Your argument is clear but lacks strong supporting details.', 'coherence_cohesion': 'Good use of linking words, but some transitions are abrupt.', 'lexical_resource': 'Vocabulary range is adequate but lacks variety.', 'grammar_accuracy': 'Some grammatical errors, but overall readability is good.'})

In [9]:
# from pydantic import BaseModel, Field, conlist
# from openai import OpenAI

# client = OpenAI()

# # Define Pydantic models
# class Step(BaseModel):
#     explanation: str
#     output: str

# class IELTSCalculation(BaseModel):
#     steps: list[Step]
#     final_band: float = Field(..., description="Final IELTS band score rounded according to IELTS rounding rules")

# # Function to calculate IELTS band
# def calculate_ielts_band(scores: list[float]) -> float:
#     average = sum(scores) / len(scores)
#     rounded_band = round(average * 2) / 2  # IELTS rounding rule: nearest 0.5 increment
#     return rounded_band

# # Example input
# scores = [7.5, 6.5, 8.0, 7.0]  # Listening, Reading, Writing, Speaking

# # Generate steps
# steps = [
#     Step(explanation="Calculate the average score across four sections.", output=str(sum(scores) / len(scores))),
#     Step(explanation="Apply IELTS rounding rules (round to nearest 0.5).", output=str(calculate_ielts_band(scores))),
# ]

# # Call OpenAI API
# completion = client.beta.chat.completions.parse(
#     model="gpt-4o-2024-08-06",
#     messages=[
#         {"role": "system", "content": "You are an IELTS scoring assistant. Calculate the IELTS band score step by step."},
#         {"role": "user", "content": f"Calculate the IELTS band score for the following section scores: {scores}"}
#     ],
#     response_format=IELTSCalculation,
# )

# ielts_calculation = completion.choices[0].message.parsed

# print(ielts_calculation)


steps=[Step(explanation='To find the overall IELTS band score, the sum of the four individual section scores is calculated.', output='Sum of scores: 7.5 + 6.5 + 8.0 + 7.0 = 29.0'), Step(explanation='The overall score is calculated by taking the average of the sum of the section scores. This is done by dividing the total sum by the number of sections, which is 4.', output='Average score: 29.0 / 4 = 7.25'), Step(explanation='The overall IELTS band score is rounded to the nearest whole or half band score. According to the rounding rules, scores ending in .25 are rounded up to the next half band.', output='Rounded score: 7.5')] final_band=7.5


In [20]:
from pydantic import BaseModel, Field
from typing import List, Dict
import math
from openai import OpenAI

client = OpenAI()

# Define Pydantic models
class Score(BaseModel):
    overall_band: float
    task_response: float
    coherence_and_cohesion: float
    lexical_resource: float
    grammatical_range_and_accuracy: float

class Feedback(BaseModel):
    task_response: str
    coherence_and_cohesion: str
    lexical_resource: str
    grammatical_range_and_accuracy: str

class IELTSWritingEvaluation(BaseModel):
    word_count: int
    score: Score
    feedback: Feedback
    suggestions: List[str]

    @classmethod
    def from_essay(
        cls, 
        essay_text: str, 
        scores: Dict[str, float], 
        feedback: Dict[str, str], 
        suggestions: List[str]
    ) -> "IELTSWritingEvaluation":
        """Factory method to compute word count and overall band dynamically"""
        word_count = len(essay_text.split())

        # Compute overall band score using IELTS rounding rules
        avg_score = sum(scores.values()) / len(scores)
        decimal_part = avg_score - math.floor(avg_score)

        if decimal_part < 0.25:
            overall_band = math.floor(avg_score)
        elif decimal_part >= 0.75:
            overall_band = math.ceil(avg_score)
        else:
            overall_band = math.floor(avg_score) + 0.5

        return cls(
            word_count=word_count,
            score=Score(overall_band=overall_band, **scores),
            feedback=Feedback(**feedback),
            suggestions=suggestions
        )

# Example essay input
# essay_text = """In modern society, the use of technology has greatly influenced the way people communicate. 
# While some argue that it has improved connections, others believe it has led to a decline in face-to-face interactions. 
# In my opinion, although technology has enhanced global communication, it has also contributed to social isolation in some cases.
# One significant advantage of technology is its ability to connect people across the world. 
# With the rise of social media and instant messaging, individuals can stay in touch regardless of geographical barriers. 
# For example, many people maintain friendships and professional relationships through platforms like WhatsApp and LinkedIn. 
# However, excessive reliance on digital communication has reduced in-person conversations. 
# Many individuals, especially younger generations, prefer texting over talking in real life, which may weaken social bonds.
# To conclude, while technology has bridged communication gaps on a global scale, it is essential to balance online and offline interactions. 
# People should make conscious efforts to engage in face-to-face conversations to maintain strong social relationships."""

essay_text = "nice to meet you, I am anh"

# Call OpenAI API for scoring, feedback, and suggestions
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an IELTS writing evaluator. Analyze the essay and provide structured scores, feedback, and suggestions."},
        {"role": "user", "content": f"Evaluate the following IELTS essay:\n{essay_text}"}
    ],
    response_format=IELTSWritingEvaluation,
)

ielts_result = completion.choices[0].message.parsed

# Construct final response
response = {
    "choices": [
        {
            "message": {
                "parsed": ielts_result.model_dump()
            }
        }
    ]
}

print(response)


{'choices': [{'message': {'parsed': {'word_count': 7, 'score': {'overall_band': 1.0, 'task_response': 1.0, 'coherence_and_cohesion': 1.0, 'lexical_resource': 1.0, 'grammatical_range_and_accuracy': 1.0}, 'feedback': {'task_response': 'The response does not address any task. It simply contains a greeting and an introduction, which is not appropriate for an IELTS essay. The essay task is not fulfilled at all.', 'coherence_and_cohesion': 'There are no structural elements of an essay present. The response does not exhibit any logical progression of ideas, paragraphs, or cohesive devices.', 'lexical_resource': 'The lexical resource is extremely limited, with only a simple greeting phrase and personal introduction. It does not demonstrate an ability to use a range of vocabulary.', 'grammatical_range_and_accuracy': 'The grammatical range is very limited, consisting only of a greeting and introduction. No attempts are made at complex or varied sentence structures.'}, 'suggestions': ['Review IEL

In [21]:
response

{'choices': [{'message': {'parsed': {'word_count': 7,
     'score': {'overall_band': 1.0,
      'task_response': 1.0,
      'coherence_and_cohesion': 1.0,
      'lexical_resource': 1.0,
      'grammatical_range_and_accuracy': 1.0},
     'feedback': {'task_response': 'The response does not address any task. It simply contains a greeting and an introduction, which is not appropriate for an IELTS essay. The essay task is not fulfilled at all.',
      'coherence_and_cohesion': 'There are no structural elements of an essay present. The response does not exhibit any logical progression of ideas, paragraphs, or cohesive devices.',
      'lexical_resource': 'The lexical resource is extremely limited, with only a simple greeting phrase and personal introduction. It does not demonstrate an ability to use a range of vocabulary.',
      'grammatical_range_and_accuracy': 'The grammatical range is very limited, consisting only of a greeting and introduction. No attempts are made at complex or varied

In [ ]:
vision_response =  client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """You are an advanced IELTS evaluator specializing in text extraction from images.
                Your task is to accurately extract the IELTS Writing Task 2 topic and essay
                from the provided image, ensuring the response follows a strict JSON format with No additional explanations, comments, or formatting outside of JSON """},
                {"role": "user", "content": [
                    {"type": "text", "text": """"Analyze the provided image and extract:\n"
                        1. The **IELTS Writing Task 2 topic** (if explicitly present in the image).\n
                        2. The **full essay response** (ensuring all text is captured accurately).\n\n
                        If the topic is **not explicitly stated**, generate a best-fit topic based on the essay content.\n\n
                        Strictly return JSON format **without markdown or extra text**:\n
                        {\n
                          \topic\: \Extracted topic here\,\n
                          \essay\: \Extracted essay text here\\n
                        }\n\n
                        
                        rule:\n
                        - No additional explanations, comments, or formatting outside of JSON.\n
                        - The topic is concise and relevant.\n
                        - The essay is fully extracted without truncation.
                     """},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                ]}
            ]
        )